# Imports

In [1]:
!pip install transformers
!pip install semanticscholar
!pip install metapub
!pip install sentencepiece
!pip install sacremoses
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import zipfile as zf
from glob import glob
# from keras.saving.hdf5_format import save_attributes_to_hdf5_group
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration, AutoTokenizer, TFBertModel, AutoModel, BertTokenizer, TFBertForQuestionAnswering
import sentencepiece
from metapub import PubMedFetcher
from semanticscholar import SemanticScholar
from metapub import FindIt
import requests
import urllib
import json
from transformers import BioGptModel, BioGptConfig, BioGptTokenizer, BioGptForCausalLM

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
2023-06-16 18:36:10 95c40d20f5ea metapub.config[5149] WARNING NCBI_API_KEY was not set.


In [2]:
# !pip install datsets transformers[sentencepiece]

In [3]:
from google.colab import drive
from google.colab import files
import os
import glob

In [4]:
# Mount google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Paths
main_path = "/content/drive/MyDrive/"
capstone_directory = main_path + "w210_Capstone_Therapeutic_Accelerator/"
starter_data = capstone_directory + "data"


In [6]:
os.listdir(starter_data)

['Mitochondrial N formyl methionine peptides contribute to exaggerated neutrophil activation in patients with COVID 19.pdf',
 'text.txt',
 'Data Post to postgresql.ipynb',
 'Connect to postgresql.ipynb',
 'Semantic Scholar and PubMed API.ipynb',
 'final.csv']

In [7]:
attributes_csv = starter_data + "/final.csv"
attributes_df = pd.read_csv(attributes_csv)
attributes_df.head(10)

,Unnamed: 0,corpusid,externalids,url,title,authors,venue,publicationvenueid,year,referencecount,citationcount,influentialcitationcount,isopenaccess,s2fieldsofstudy,publicationtypes,publicationdate,journal,updated,id
0,0,52273203,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/39063bcd...,Large conformational changes of a highly dynam...,"{""{\""authorId\"": \""51285510\"", \""name\"": \""Isa...",Communications Biology,069e05e7-ca35-41d0-a8c7-bdc9ec6a82af,2018.0,28,13,0,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...",{JournalArticle},2018-09-03,"{""name"": ""Communications Biology"", ""pages"": nu...",2023-03-31T01:28:37.897Z,5692596
1,1,23708908,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/6ec7c156...,Drug sensitivity profiling identifies potentia...,"{""{\""authorId\"": \""5699001\"", \""name\"": \""H. K...",OncoTarget,106ecc86-1801-4b36-a05c-d36db69a968c,2017.0,46,24,0,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...",{JournalArticle},2017-10-31,"{""name"": ""Oncotarget"", ""pages"": ""97516 - 97527...",2023-04-05T06:18:15.837Z,5692609
2,2,13232625,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/c8566272...,Men's memory for women's sexual-interest and r...,"{""{\""authorId\"": \""4425040\"", \""name\"": \""T. T...",NaN,NaN,2011.0,36,13,0,True,"{""{\""category\"": \""Psychology\"", \""source\"": \...",{Review},2011-09-01,"{""name"": ""Applied Cognitive Psychology"", ""page...",2023-04-05T08:18:02.240Z,5692611
3,3,73484844,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/d4c9b2fa...,The Reactive Plasticity of Hippocampal Ionotro...,"{""{\""authorId\"": \""2225106\"", \""name\"": \""A. M...",International Journal of Molecular Sciences,8506a01a-40b8-4e6f-bbb8-ce2492139c15,2019.0,97,15,1,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...","{Review,JournalArticle}",2019-02-27,"{""name"": ""International Journal of Molecular S...",2023-05-20T17:20:54.230Z,5692615
4,4,229159752,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/7f13abe2...,High Mortality Rate in Adult COVID-19 Inpatien...,"{""{\""authorId\"": \""7685173\"", \""name\"": \""Saee...",Journal of Multidisciplinary Healthcare,12e5e550-28bd-401c-a348-f347956deec4,2020.0,37,17,1,True,"{""{\""category\"": \""Medicine\"", \""source\"": \""s...","{JournalArticle,Review}",2020-12-01,"{""name"": ""Journal of Multidisciplinary Healthc...",2023-05-23T13:34:07.581Z,5692620
5,5,10041098,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/92b26dee...,NADH-dependent metabolism of nitric oxide in a...,"{""{\""authorId\"": \""1792255\"", \""name\"": \""A. I...",Planta,66507f0b-12a1-4058-a35c-73029853d5b9,2004.0,30,145,3,False,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...",{JournalArticle},2004-01-22,"{""name"": ""Planta"", ""pages"": ""95-102"", ""volume""...",2023-04-05T15:01:41.273Z,5692625
6,6,19472375,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/2c682757...,Aspects of posttraumatic stress disorder in lo...,"{""{\""authorId\"": \""2000616\"", \""name\"": \""A. D...",Journal of cancer survivorship,649b7693-cb10-4892-b8a2-65b9b51fcd50,2016.0,33,18,0,False,"{""{\""category\"": \""Psychology\"", \""source\"": \...",{JournalArticle},2016-02-26,"{""name"": ""Journal of Cancer Survivorship"", ""pa...",2023-04-05T19:07:12.948Z,5692626
7,7,219603447,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/9b9659a4...,Multicenter assessment of the Brain Injury Gui...,"{""{\""authorId\"": \""145776250\"", \""name\"": \""Ab...",Trauma Surgery & Acute Care Open,d40f33e8-6f79-4e91-a62e-acf29f9a3e2f,2020.0,35,11,0,True,"{""{\""category\"": \""Medicine\"", \""source\""

## Load Models & Tokenizers

In [8]:
bio_bert_model = AutoModel.from_pretrained("gsarti/biobert-nli")
bio_bert_tokenizer = AutoTokenizer.from_pretrained("gsarti/biobert-nli")
original_bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
T5Abstract_model = TFT5ForConditionalGeneration.from_pretrained('t5-base')
T5tokens = T5Tokenizer.from_pretrained('t5-base')


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [10]:
# Get info about the latest release
latest_release = requests.get("http://api.semanticscholar.org/datasets/v1/release/2023-05-30",
                              headers={'x-api-key':'jvN8XitMY59Wtoqm3FZnc94qRHnsKw3z3hLNh7XB'}).json()

# Get info about past releases
dataset_ids = requests.get("http://api.semanticscholar.org/datasets/v1/release").json()
earliest_release = requests.get(f"http://api.semanticscholar.org/datasets/v1/release/{dataset_ids[0]}").json()

# Get info about the papers dataset
papers = requests.get("http://api.semanticscholar.org/datasets/v1/release/latest/dataset/papers",
                      headers={'x-api-key':'jvN8XitMY59Wtoqm3FZnc94qRHnsKw3z3hLNh7XB'}).json()

In [11]:
print(json.dumps(latest_release['datasets'][7], indent=1))

{
 "name": "s2orc",
 "description": "Full-body paper text parsed from open-access PDFs. Identifies structural elements such as paragraphs, sections, and bibliography entries.\n5M records in 30 4GB files.",
 "README": "Semantic Scholar Academic Graph Datasets\n\nThe \"s2orc\" dataset contains parsed full-body text from selected papers.\n\nA subset of this data was previously released (in a different format) as S2ORC https://github.com/allenai/s2orc\n\nThe body text is parsed from PDF documents using Grobid, documented at https://grobid.readthedocs.io.\nIts output is converted from XML into a single string with a set of annotation spans.\n\nSCHEMA\n - externalIds: IDs of this paper in different catalogs\n - content:\n   - source:\n\t   - pdfUrls: URLs to the PDF\n\t   - oaInfo: license/url/status information from Unpaywall\n   - text: Full body text as a single string\n   - annotations: Annotated spans of the full body text\n\n\nLICENSE\nThis collection is licensed under ODC-BY. (https:/

In [12]:
sch = SemanticScholar()

sch = SemanticScholar(api_key='jvN8XitMY59Wtoqm3FZnc94qRHnsKw3z3hLNh7XB')

paper = sch.get_paper('d0bc1501ae6f54dd16534e651d90d2aeeeb1cfc1')  #'10.1093/mind/lix.236.433')

paper.referenceCount #number of references in this one, can help us create some sort of pagerank type algo
paper.citationCount #number of papers that cited this one
paper.externalIds
text_example = paper.abstract
# paper.embedding
print(text_example)

In spite of many years of work by a multitude of organizations, a clear and simple standard for software engineering and management requirements and a method to assess their applicability to projects of various types and sizes remains elusive. From IEEE to CMMI to NASA's NPR 7150.2, there is no shortage of sources of information providing various types of requirements and standards for software engineering. Even a book on software project management for “dummies” approaches 400 pages. Wading through this information can dizzy the mind of even the most experienced software engineer; the newbie just trying to “do the right thing” will probably give up, open a text editor and start coding. This lack of clarity and simplicity perhaps goes a long way towards explaining why, in spite of this large body of work, there remains such an incredible variability in the knowledge and application of software engineering discipline not only from one organization to the next, but between groups within 

In [53]:
# Get info about the papers dataset
papers_2 = requests.get("http://api.semanticscholar.org/datasets/v1/release/latest/dataset/papers/5692596",
                      headers={'x-api-key':'jvN8XitMY59Wtoqm3FZnc94qRHnsKw3z3hLNh7XB'}).json()
papers_2

{'error': 'No dataset named papers/5692596 in release latest'}

In [13]:
max_sequence_length = 512
embedding_size = 200

In [14]:
T5tokens(text_example)

{'input_ids': [86, 17912, 13, 186, 203, 13, 161, 57, 3, 9, 16635, 13, 2371, 6, 3, 9, 964, 11, 650, 1068, 21, 889, 3867, 11, 758, 1502, 11, 3, 9, 1573, 12, 6570, 70, 1120, 2176, 2020, 12, 1195, 13, 796, 1308, 11, 4342, 3048, 3, 29748, 5, 1029, 23570, 12, 3, 5518, 7075, 12, 15971, 31, 7, 445, 5554, 489, 1808, 18189, 6, 132, 19, 150, 16316, 13, 2836, 13, 251, 1260, 796, 1308, 13, 1502, 11, 2443, 21, 889, 3867, 5, 1441, 3, 9, 484, 30, 889, 516, 758, 21, 105, 26, 30885, 153, 6315, 4837, 1688, 5, 3129, 26, 53, 190, 48, 251, 54, 1227, 19974, 8, 809, 13, 237, 8, 167, 1906, 889, 9739, 117, 8, 126, 4232, 131, 1119, 12, 105, 26, 32, 8, 269, 589, 153, 56, 1077, 428, 95, 6, 539, 3, 9, 1499, 6005, 11, 456, 3, 9886, 5, 100, 2136, 13, 12428, 11, 16538, 2361, 1550, 3, 9, 307, 194, 1587, 13834, 572, 6, 16, 17912, 13, 48, 508, 643, 13, 161, 6, 132, 3048, 224, 46, 3904, 27980, 16, 8, 1103, 11, 917, 13, 889, 3867, 7998, 59, 163, 45, 80, 1470, 12, 8, 416, 6, 68, 344, 1637, 441, 8, 337, 1470, 6, 42, 237, 344

In [15]:
### Instead of summarization can be used for classification of papers
def biobert_classifier(embedding_size = 200, input_dimensions = 3, hidden_layers = 0, max_sequence_length = 512, learning_rate=0.01):
  input_ids = tf.keras.layers.Input(shape=embedding_size, name="input_ids")
  token_type_ids = tf.keras.layers.Input(shape=embedding_size, name="token_type_id")
  attention_mask = tf.keras.layers.Input(shape=embedding_size, name="attention_mask")

  model_inputs = {"input_ids": input_ids,
                  "token_type_ids": token_type_ids,
                  "attention_mask": attention_mask}

  embedding_matrix = tf.keras.layers.Embedding(200)

  normalization_layer = tf.keras.layers.BatchNormalization()

  attention_layer = tf.keras.layers.Attention()

  pooler_layer = bio_bert_model(model_inputs)[0]

  dense_layer = tf.keras.layers.Dense(100, activation="relu")(pooler_layer)

  dropout_layer = tf.keras.layers.Dropout(0.3)(dense_layer)

  final_layer = tf.keras.layers.Dense(1, activation="relu")(dropout_layer)

  classification_layer = tf.keras.layers.Dense(1, activation='sigmoid')(final_layer)

  model = tf.keras.Model(inputs = [input_ids, token_type_ids, attention_mask], outputs = [classification_layer])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                loss = tf.keras.losses.BinaryCrossentropy,
                metrics = [tf.keras.metrics.Accuracy, tf.keras.metrics.Precision])

  return model

In [16]:
summary_task_prefix = "Summarize :"
qa_task_prefix = "Question :"

In [17]:
### T5 Abstractive Text Summarization Model
def t5summary_model(tokenizer, text, t5model):
    summarize = "summarize: "
    encoding = tokenizer([summarize+text], return_tensors='tf')
    output = t5model.generate(encoding.input_ids,
                             num_beams=3,
                            no_repeat_ngram_size=2,
                              top_k=10,
                              top_p=80,
                              max_length=50,
                              min_length=30
                             )
    print([tokenizer.decode(w, skip_special_tokens=True, clean_up_tokenization_spaces=True) for w in output])
#     print(summary)

In [18]:
T5Abstract_model

In [19]:
biogpttokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
biogptmodel = BioGptForCausalLM.from_pretrained("microsoft/biogpt")


In [20]:
t5summary_model(T5tokens, text_example, T5Abstract_model)

['a clear and simple standard for software engineering and management requirements remains elusive. the paper will make an admittedly bold and brash attempt to boil it all down into something anyone can understand, hopefully resulting in an brief reference']


### Q&A Model

In [21]:
bertqa = TFBertForQuestionAnswering.from_pretrained('bert-base-uncased')
berttokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
sample_question = "question: What is the capital city of Switzerland?"
sample_context = "context: Many people are surprised when they hear that pretty little Bern is the Swiss capital. Surely industrial Zurich or international Geneva would be more logical, they say. But it's precisely to avoid a concentration of power that Bern was chosen as the “federal city” exactly 170 years ago."

tokenized_qa = berttokenizer([sample_question + sample_context], return_tensors ='tf')
model_output = bertqa(**tokenized_qa)

answer_start_index = int(tf.math.argmax(model_output.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(model_output.end_logits, axis=-1)[0])
print(answer_end_index)
answer_encoded = tokenized_qa.input_ids[0,answer_start_index:answer_end_index]
start_answer = tokenized_qa.input_ids[0][answer_start_index]
print(berttokenizer.decode(answer_encoded, skip_special_tokens=True, clean_up_tokenization_spaces=True))

69
question : what is the capital city of switzerland? context : many people are surprised when they hear that pretty little bern is the swiss capital. surely industrial zurich or international geneva would be more logical, they say. but it's precisely to avoid a concentration of power that bern was chosen as the “ federal city ” exactly 170 years ago.


In [31]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

In [45]:
from transformers import RobertaForQuestionAnswering, RobertaTokenizer, AutoTokenizer, TFAutoModel
roberta_tokens = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
robertaqa_model = TFAutoModel.from_pretrained("deepset/roberta-base-squad2")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['qa_outputs.bias', 'qa_outputs.weight', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
roberta_tokenized_qa = roberta_tokens([sample_question + sample_context], padding=True,  return_tensors='tf')
roberta_model_layer = robertaqa_model(**roberta_tokenized_qa)



AttributeError: ignored

In [30]:
tokenized_qa.input_ids[0]

<tf.Tensor: shape=(70,), dtype=int32, numpy=
array([  101,  3160,  1024,  2054,  2003,  1996,  3007,  2103,  1997,
        5288,  1029,  6123,  1024,  2116,  2111,  2024,  4527,  2043,
        2027,  2963,  2008,  3492,  2210, 16595,  2003,  1996,  5364,
        3007,  1012,  7543,  3919, 10204,  2030,  2248,  9810,  2052,
        2022,  2062, 11177,  1010,  2027,  2360,  1012,  2021,  2009,
        1005,  1055, 10785,  2000,  4468,  1037,  6693,  1997,  2373,
        2008, 16595,  2001,  4217,  2004,  1996,  1523,  2976,  2103,
        1524,  3599, 10894,  2086,  3283,  1012,   102], dtype=int32)>

In [56]:
model_output

TFQuestionAnsweringModelOutput(loss=None, start_logits=<tf.Tensor: shape=(1, 67), dtype=float32, numpy=
array([[ 0.16508265,  0.45413932,  0.04580224,  0.09424645, -0.07182357,
        -0.11457947,  0.11367559,  0.12410395, -0.00738204,  0.29390356,
        -0.19900739, -0.12262025, -0.00352622, -0.26001504,  0.16652006,
         0.11015014, -0.05808127, -0.07685345, -0.05458946,  0.0009411 ,
        -0.01198626, -0.01611032, -0.11756027, -0.08084115,  0.04347808,
        -0.48372173, -0.09519623, -0.24002753,  0.12300973,  0.3441984 ,
         0.14760727,  0.06125803, -0.20339994,  0.13757387,  0.00848398,
        -0.21722472,  0.04151647, -0.19865523, -0.18272205, -0.48353076,
         0.01219941,  0.0961633 , -0.25447947, -0.24058512,  0.04575343,
        -0.48203897, -0.21114022, -0.5456536 , -0.5361859 , -0.62731385,
        -0.24357261, -0.1540298 ,  0.04120081,  0.18258283, -0.15227222,
        -0.01945501,  0.08738828, -0.1486671 ,  0.01810225, -0.10272819,
        -0.07920434,

In [41]:
### BioBERT or BERT Q&A or Clincal-T5-Large
sample_question = "question: What is the capital city of Switzerland?"
sample_context = "context: Many people are surprised when they hear that pretty little Bern is the Swiss capital. Surely industrial Zurich or international Geneva would be more logical, they say. But it's precisely to avoid a concentration of power that Bern was chosen as the “federal city” exactly 170 years ago."

encoding_gpt = biogpttokenizer([sample_question + sample_context], return_tensors='tf')
encoding_gpt
# encoded_answer = biogptmodel.generate(encoding_gpt.input_ids,
                                      # num_beams = 3,
                                      # top_p = 80,
                                      # top_k = 4,
                                      # no_repeat_ngram_size = 2,
                                      # max_length = 10,
                                      # min_length =2
                                      # )
# decoded_answer = biogpttokenizer.decode(encoded_answer)
# print(decoded_answer)

{'input_ids': <tf.Tensor: shape=(1, 79), dtype=int32, numpy=
array([[    2,  2969,    20,  5732,    21,     6, 11991,  5712,     5,
        14693,   927,  1544,    20,  2621,   962,    31, 32991,  4464,
          154,   298,  3734,  1770,    22,   968,   765,  7484,  1161,
        14421,   169,    21,     6, 11029, 11991,     4,   250, 37144,
         5132,  1196,  1595, 10956,    30,  3560,  7661,  3838,    14,
          739,    33,    77, 10775,     7,   298, 20933,     4,  6349,
          114,   121,  8641,    13,  3350,    14,   276,     5,  1382,
           22, 14421,   169,    17,  4852,    27,     6,     3, 14191,
         5712,     3, 15362,  8093,   113,  5306,     4]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 79), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1,

### Extractive Summary Models

In [22]:
### T5 or T5v1 or Clincal-T5-Large or Bio-GPT

### Classification Model

In [23]:
### BERT or Bio-GPT

### NER Model

In [24]:
### BioELECTRA